In [1]:
import gym
env = gym.make('CartPole-v1')

In [2]:
import mlflow

# connect mlflow client to the mlflow server that runs on localhost:5000
MLFLOW_SERVER_URI = 'http://localhost:5000'
mlflow.set_tracking_uri(str(MLFLOW_SERVER_URI))

EXPERIMENT_NAME = 'DQN_24_02_08'
mlflow.set_experiment(EXPERIMENT_NAME)

In [ ]:
import optuna

from config import OPTUNA_DB

study = optuna.create_study(
    study_name=EXPERIMENT_NAME,
    direction='maximize',
    load_if_exists=True,
    storage=f'sqlite:///{OPTUNA_DB}'
)

In [ ]:
optuna.logging.set_verbosity(optuna.logging.ERROR)

A ideia é treinar cada conjunto de hiperparâmetros por 200 épocas e avaliar por 1000, a função retorna a recompensa média dessas 1000 iterações

In [ ]:
from optimize_hyperparameters import objective

# we define a lambda function because study.optimize()
# expect the objective function to have only 1 input
# (trial), while our objective function hast 2 extra
# inputs I defined to add flexibility to the script
func = lambda trial: objective(trial,
                               force_linear_model=False,
                               n_episodes_to_train=200)

class CheckHyperparamMeanRewardThreshold:
    def __init__(self, reward_threshold: float):
        self.reward_threshold = reward_threshold

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial) -> None:
        if trial.value is not None and trial.value >= self.reward_threshold:
            print((f'Stopping hyperparameter search because trial.value ({trial.value}) '
                   f'hit threshold ({self.reward_threshold})'))
            study.stop()

# Stop hyperparameter search when we hit a perfect mean reward of 500
hyperparam_search_stop_callback = CheckHyperparamMeanRewardThreshold(500.0)

study.optimize(func, n_trials=1000, callbacks=[hyperparam_search_stop_callback], show_progress_bar=True)
# Existe um puta problema relacionado ao como representar redes com múltiplas camadas, não estou com saco pra resolver isso agora
# mas saiba que existe e que pra resolver precisa dissecar o dicionário de parâmetros que o optuna usa


In [ ]:
best_trial = study.best_trial

hparams = {k: best_trial.params[k] for k in best_trial.params if k != 'seed'}
#hparams['nn_hidden_layers'] = eval(hparams['nn_hidden_layers']) 
print(hparams)

SEED = best_trial.params['seed']
print('Seed: ', SEED)

In [ ]:
import os
from q_agent import QAgent
from config import SAVED_AGENTS_DIR

diretorio_pai = SAVED_AGENTS_DIR / 'CartPole-v1'

# Lista de todos os diretórios no diretório_pai
diretorios = [d for d in os.listdir(diretorio_pai) if os.path.isdir(os.path.join(diretorio_pai, d))]

# Função para obter o tempo de criação de um diretório
def obter_tempo_criacao(diretorio):
    caminho_completo = os.path.join(diretorio_pai, diretorio)
    return os.path.getctime(caminho_completo)

# Encontrar o último diretório criado
ultimo_diretorio = max(diretorios, key=obter_tempo_criacao)

# Caminho completo para o último diretório criado
path_to_saved_model = diretorio_pai / str(ultimo_diretorio)

agent = QAgent.load_from_disk(env, path_to_saved_model)

In [ ]:
from loops import evaluate
rewards, steps = evaluate(
    agent, env,
    n_episodes=1000,
    epsilon=0.00
)

In [ ]:
plt.plot(rewards, label='Rewards per Episode')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Rewards per Episode')
plt.legend()
plt.show()

In [ ]:
import numpy as np
reward_avg = np.array(rewards).mean()
reward_std = np.array(rewards).std()
print(f'Reward average {reward_avg:.2f}, std {reward_std:.2f}')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

fig, ax = plt.subplots(figsize = (10, 4))
ax.set_title("Rewards")    
pd.Series(rewards).plot(kind='hist', bins=100)

plt.show()